# Polars 

## Polars란?
- 구조화된 데이터를 다루기 위한 아주 빠른 데이터프레임 라이브러리
- 코어가 Rust로 구현되어 있어 빠름
- Python, R 및 NodeJS에서 사용 가능

## 핵심 기능
### Fast
- Rust로 작성되어 빠름
- 외부 의존성 없이 기계에 가깝게 설계

### I/O
- 일반적인 데이터 스토리지(local, Cloud Storage & Database)를 위한 최우선 지원

### 직관적인 API
- 쿼리를 의도한대로 작성해도, Polars는 내부적으로 내장 쿼리 옵티마이저를 사용하여 가장 효율적인 방법을 선택

### Out of Core
- 스트림 API는 메모리에 모든 데이터를 메모리에 넣지 않아도 데이터를 처리할 수 있게 해줌

### 병렬성
- 추가적인 구성 없이 사용가능한 모든 CPU코어를 활용해 workload를 분산 처리

### 벡터 쿼리 엔진

### GPU Support
- in-memory에서 최대한의 성능을 위해 선택적으로 NVIDIA GPUs 사용

### Apache Arrow 지원
- zero-copy를 활용하여 Arrow data를 consume/produce 가능
- polars가 pyarrow/arrow 구현체를 기반하진 않았으나, 자체 compute 및 buffer 구현이 있음

## 철학
- polars의 목표는 가볍고 빠른 데이터프레임 라이브러리를 제공하는 것
    - 모든 코어 활용
    - 불필요한 workload와 memory를 줄이기 위해 쿼리 최적화
    - 사용가능한 RAM보다 더 큰 데이터셋은 핸들링
    - 일관되고 예측가능한 API
    - 엄격한 스키마 준수(쿼리 전 데이터 타입을 알아야 함수
- Rush로 작성되어 C/C++ 수준의 성능을 제공하며 쿼리엔진에서 성능이 중요한 부분을 완전히 제어



## Polars 설치
### pip 
`pip install polars`
### poetry
`poetry add polars`   
현재 버전(^1.24) 기준 python3.9 이상부터 지원




## Polars 설치
### pip 
`pip install polars`
### poetry
`poetry add polars`   
현재 버전(^1.24) 기준 python3.9 이상부터 지원




## Polars 사용
### 작은 데이터프레임 생성 후 write / read csv

In [2]:
import polars as pl
import datetime as dt

In [3]:
df = pl.DataFrame(
    {
        "name": ["Alice Archer", "Ben Brown", "Chloe Cooper", "Daniel Donovan"],
        "birthdate": [
            dt.date(1997, 1, 10),
            dt.date(1985, 2, 15),
            dt.date(1983, 3, 22),
            dt.date(1981, 4, 30),
        ],
        "weight": [57.9, 72.5, 53.6, 83.1],  # (kg)
        "height": [1.56, 1.77, 1.65, 1.75],  # (m)
    }
)

df

name,birthdate,weight,height
str,date,f64,f64
"""Alice Archer""",1997-01-10,57.9,1.56
"""Ben Brown""",1985-02-15,72.5,1.77
"""Chloe Cooper""",1983-03-22,53.6,1.65
"""Daniel Donovan""",1981-04-30,83.1,1.75


In [5]:
import os 
os.makedirs("../data", exist_ok=True)
df.write_csv("../data/sample.csv")


df = pl.read_csv("../data/sample.csv", try_parse_dates=True) # try_parse_dates=True 날짜 형식 자동 인식
df



name,birthdate,weight,height
str,date,f64,f64
"""Alice Archer""",1997-01-10,57.9,1.56
"""Ben Brown""",1985-02-15,72.5,1.77
"""Chloe Cooper""",1983-03-22,53.6,1.65
"""Daniel Donovan""",1981-04-30,83.1,1.75


### 표현식
- 표현식은 Polars의 주요 강점 중 하나 
- 데이터의 변환을 표현하는 유연하고 모듈식을 제공

In [7]:
# 표현식의 예시
pl.col("weight") / (pl.col("height") ** 2)

<Expr ['[(col("weight")) / (col("heigh…'] at 0x107C99C10>

- 이 표현식은 컬럼명 weight을 height의 제곱으로 나누는 것(BMI 계산)
- 이 코드는 추상적인 계산을 표현하고, Polars의 Context 내부에서 결과를 시리즈로 구체화 시킴
- 아래는 다른 context안에서의 Polars 표현식의 예시
    - select
    - with_columns
    - filter
    - group_by

#### SELECT
- context `SELECT`는 데이터프레임의 컬럼들에 대한 select와 조작을 하게 해줌
- 간단한 예시로, 제공한 각각의 표현식이 결과 데이터프레임의 컬럼에 적용의

In [8]:
result = df.select(
    pl.col("name"),
    pl.col("birthdate").dt.year().alias("birth_year"),
    (pl.col("weight") / (pl.col("height") ** 2)).alias("bmi"),
)
result

name,birth_year,bmi
str,i32,f64
"""Alice Archer""",1997,23.791913
"""Ben Brown""",1985,23.141498
"""Chloe Cooper""",1983,19.687787
"""Daniel Donovan""",1981,27.134694


- Polars는 표현확장이라고 불리는 기능을 제공하는데, 하나의 표현식이 여러 개의 표현식의 약어 기능을 할 수 있음
- 아래 예시에선 하나의 표현식으로 weight과 height 두 컬럼을 조작(표현확장을 사용)
- 표현확장을 사용하며 `.name.suffix`를 사용하여 원본 컬럼명에 suffix들을 추가할 것

In [9]:
result = df.select(
    pl.col("name"),
    (pl.col("weight", "height") * 0.95).round(2).name.suffix("-5%"),
)
result

name,weight-5%,height-5%
str,f64,f64
"""Alice Archer""",55.01,1.48
"""Ben Brown""",68.88,1.68
"""Chloe Cooper""",50.92,1.57
"""Daniel Donovan""",78.94,1.66


#### WITH_COLUMNS
- select와 유사하나 컬럼을 선택하는 것이 아닌 컬럼을 추가하는 것
- 이 컬럼들은 표현식을 통해 생성되고, 이름을 지정할 수 있음
- 아래 예시는 기존 컬럼에 birthday와 bmi 컬럼을 추가

In [10]:
result = df.with_columns(
    birth_year=pl.col("birthdate").dt.year(),
    bmi=pl.col("weight") / (pl.col("height") ** 2),
)

result


name,birthdate,weight,height,birth_year,bmi
str,date,f64,f64,i32,f64
"""Alice Archer""",1997-01-10,57.9,1.56,1997,23.791913
"""Ben Brown""",1985-02-15,72.5,1.77,1985,23.141498
"""Chloe Cooper""",1983-03-22,53.6,1.65,1983,19.687787
"""Daniel Donovan""",1981-04-30,83.1,1.75,1981,27.134694


- 위의 예시에서 컬럼명을 결정할 때 alias를 쓰는 것이 아닌 이름 표현식을 사용
- select와 group_by와 같은 다른 context도 이름 표현식을 허용

#### FILTER
- filter는 원본에서 하위 집합으로 두번 째 데이터프레임을 만들 수 있음

In [12]:
result = df.filter(pl.col("birthdate").dt.year() < 1990)
result

name,birthdate,weight,height
str,date,f64,f64
"""Ben Brown""",1985-02-15,72.5,1.77
"""Chloe Cooper""",1983-03-22,53.6,1.65
"""Daniel Donovan""",1981-04-30,83.1,1.75


- 여러 조건 표현식을 분리된 parameters를 사용하여 제공할 수 있음
- 이는 `&` 연산자를 사용하여 다 같이 놓는 것 보다 편리함

In [13]:
result = df.filter(
    pl.col("birthdate").is_between(dt.date(1982, 12, 31), dt.date(1996, 1, 1)),
    pl.col("height") > 1.7,
)
result


name,birthdate,weight,height
str,date,f64,f64
"""Ben Brown""",1985-02-15,72.5,1.77


#### GROUP BY
- group_by를 사용하면 하나 이상의 표현식에서 같은 값을 가지는 데이터 프레임의 행을 그룹화
- 아래 에시는 10년 별로 사람들이 얼마나 태어났는 지에 대한 숫자

In [14]:
result = df.group_by(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
    maintain_order=True,
).len()
result


decade,len
i32,u32
1990,1
1980,3


- key 요소인 `maintain_order`는 Polars가 원본 데이터프레임와 동일한 순서로 결과 그룹을 표시하도록 강제함
- 그룹 오퍼레이팅을 느리게 하지만 예시이기에 확실한 재현성을 위해 사용
- `group_by` context 사용 이후 agg를 사용하여 집계연산을 결과에 사용할 수 있음

In [15]:
result = df.group_by(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
    maintain_order=True,
).agg(
    pl.len().alias("sample_size"),
    pl.col("weight").mean().round(2).alias("avg_weight"),
    pl.col("height").max().alias("tallest"),
)
result

decade,sample_size,avg_weight,tallest
i32,u32,f64,f64
1990,1,57.9,1.56
1980,3,69.73,1.77


## 데이터프레임 결합
- Polars는 두 개의 데이터프레임을 결합하기 위한 몇가지 도구를 제공
- 여기에선 join과 concatenation 사용
### Join
- Polars는 많은 join 알고리즘을 제공
- 아래 예시에선 이름을 사용하여 left outer join

In [16]:
df2 = pl.DataFrame(
    {
        "name": ["Ben Brown", "Daniel Donovan", "Alice Archer", "Chloe Cooper"],
        "parent": [True, False, False, False],
        "siblings": [1, 2, 3, 4],
    }
)
df2

name,parent,siblings
str,bool,i64
"""Ben Brown""",true,1
"""Daniel Donovan""",false,2
"""Alice Archer""",false,3
"""Chloe Cooper""",false,4


In [17]:
df.join(df2, on="name", how="left")

name,birthdate,weight,height,parent,siblings
str,date,f64,f64,bool,i64
"""Alice Archer""",1997-01-10,57.9,1.56,false,3
"""Ben Brown""",1985-02-15,72.5,1.77,true,1
"""Chloe Cooper""",1983-03-22,53.6,1.65,false,4
"""Daniel Donovan""",1981-04-30,83.1,1.75,false,2


### Concatenation
- 데이터프레임 결합은 메소드의 사용에 따라 더 넓거나 긴 데이터프레임을 만듬
- 예시에선 다른 사람들이 들어 있는 데이터프레임이 있을 때, 데이터프레임을 수직 결합하여 더 긴 데이터프레임을 만드는 것

In [18]:
df3 = pl.DataFrame(
    {
        "name": ["Ethan Edwards", "Fiona Foster", "Grace Gibson", "Henry Harris"],
        "birthdate": [
            dt.date(1977, 5, 10),
            dt.date(1975, 6, 23),
            dt.date(1973, 7, 22),
            dt.date(1971, 8, 3),
        ],
        "weight": [67.9, 72.5, 57.6, 93.1],  # (kg)
        "height": [1.76, 1.6, 1.66, 1.8],  # (m)
    }
)
df3

name,birthdate,weight,height
str,date,f64,f64
"""Ethan Edwards""",1977-05-10,67.9,1.76
"""Fiona Foster""",1975-06-23,72.5,1.6
"""Grace Gibson""",1973-07-22,57.6,1.66
"""Henry Harris""",1971-08-03,93.1,1.8


In [19]:
pl.concat([df, df3], how="vertical")

name,birthdate,weight,height
str,date,f64,f64
"""Alice Archer""",1997-01-10,57.9,1.56
"""Ben Brown""",1985-02-15,72.5,1.77
"""Chloe Cooper""",1983-03-22,53.6,1.65
"""Daniel Donovan""",1981-04-30,83.1,1.75
"""Ethan Edwards""",1977-05-10,67.9,1.76
"""Fiona Foster""",1975-06-23,72.5,1.6
"""Grace Gibson""",1973-07-22,57.6,1.66
"""Henry Harris""",1971-08-03,93.1,1.8
